In [5]:
import requests
from bs4 import BeautifulSoup
import re



# Jeśli chcesz pobrać zawartość ze strony internetowej, użyj tego:
url = "https://www.bip.krakow.pl/?mmi=234"
response = requests.get(url)
html_content = response.text


soup = BeautifulSoup(html_content, 'html.parser')

link_data = []

# Przechodzimy przez wszystkie znaczniki <p>
for p in soup.find_all('p'):
    # Znajdujemy wszystkie linki <a> w bieżącym <p>
    a_tags = p.find_all('a', href=True)
    
    # Jeśli w <p> jest dokładnie jeden link
    if len(a_tags) == 1:
        a = a_tags[0]
        link_data.append({
            'href': a['href'],
            'text': a.get_text(strip=True),
            'parent_text': p.get_text(strip=True),
            'full_p': str(p)  # Opcjonalnie: cała zawartość <p> jako HTML
        })

# Wyświetlamy wyniki
for item in link_data:
    print(f"Znaleziono link w pojedynczym <p>:")
    print(f"URL: {item['href']}")
    print(f"Tekst linku: {item['text']}")
    print(f"Tekst paragrafu: {item['parent_text']}")
    print("-" * 50)

Znaleziono link w pojedynczym <p>:
URL: /zalaczniki/dokumenty/n/299488/karta
Tekst linku: pdf 1,7 MB
Tekst paragrafu: IV kwartał 2020/ 4th Quarter 2020 [pdf 1,7 MB]
--------------------------------------------------
Znaleziono link w pojedynczym <p>:
URL: /zalaczniki/dokumenty/n/292549/karta
Tekst linku: [pdf 1,8 MB]
Tekst paragrafu: III kwartał 2020/ 3rd Quarter 2020[pdf 1,8 MB]
--------------------------------------------------
Znaleziono link w pojedynczym <p>:
URL: /zalaczniki/dokumenty/n/283933/karta
Tekst linku: [pdf 1,2 MB]
Tekst paragrafu: II kwartał 2020/ 1nd Quarter 2020[pdf 1,2 MB]
--------------------------------------------------
Znaleziono link w pojedynczym <p>:
URL: /zalaczniki/dokumenty/n/274429/karta
Tekst linku: [pdf 2,7 MB]
Tekst paragrafu: I kwartał 2020/ 1st Quarter 2020[pdf 2,7 MB]
--------------------------------------------------
Znaleziono link w pojedynczym <p>:
URL: /zalaczniki/dokumenty/n/265574/karta
Tekst linku: [pdf 1,6 MB]
Tekst paragrafu: IV kwartał 20

In [7]:
for link in link_data:
    url = f"https://www.bip.krakow.pl{link['href']}"
    print(url)

https://www.bip.krakow.pl/zalaczniki/dokumenty/n/299488/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/292549/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/283933/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/274429/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/265574/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/258707/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/250558/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/242357/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/232312/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/225203/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/218682/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/210593/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/202633/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/196326/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/188762/karta
https://www.bip.krakow.pl/zalaczniki/dokumenty/n/181538/karta
https://

In [11]:
base_url = "https://www.bip.krakow.pl"

download_dir = "pobrane_pliki"  # Katalog do zapisywania plików
os.makedirs(download_dir, exist_ok=True)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

session = requests.Session()

for item in link_data:
    # 1. Przejdź na stronę z linka
    page_url = urljoin(base_url, item['href'])
    try:
        print(f"Przetwarzam: {page_url}")
        response = session.get(page_url, headers=headers)
        response.raise_for_status()
        
        # 2. Znajdź przycisk "zobacz"
        soup = BeautifulSoup(response.text, 'html.parser')
        download_btn = soup.find('a', {'class': 'btn btn-primary', 'href': True}, string='zobacz')
        
        if download_btn:
            # 3. Pobierz plik PDF
            download_url = urljoin(base_url, download_btn['href'])
            print(f"Znaleziono link do pobrania: {download_url}")
            
            file_response = session.get(download_url, headers=headers, stream=True)
            file_response.raise_for_status()
            
            # Pobierz nazwę pliku z nagłówków lub URL
            content_disposition = file_response.headers.get('content-disposition')
            if content_disposition:
                filename = content_disposition.split('filename=')[1].strip('"\'')
            else:
                filename = os.path.basename(download_url.split('?')[0]) + '.pdf'
            
            # Zapisz plik
            file_path = os.path.join(download_dir, filename)
            with open(file_path, 'wb') as f:
                for chunk in file_response.iter_content(1024):
                    f.write(chunk)
            
            print(f"Pobrano plik: {file_path}")
        else:
            print("Nie znaleziono przycisku pobierania na stronie")
    
    except Exception as e:
        print(f"Błąd podczas przetwarzania {page_url}: {str(e)}")

print("Zakończono pobieranie plików")

Przetwarzam: https://www.bip.krakow.pl/zalaczniki/dokumenty/n/299488/karta
Znaleziono link do pobrania: https://www.bip.krakow.pl/plik.php?zid=299488&wer=0&new=t&mode=shw
Pobrano plik: pobrane_pliki\299488_0.pdf
Przetwarzam: https://www.bip.krakow.pl/zalaczniki/dokumenty/n/292549/karta
Znaleziono link do pobrania: https://www.bip.krakow.pl/plik.php?zid=292549&wer=0&new=t&mode=shw
Pobrano plik: pobrane_pliki\292549_0.pdf
Przetwarzam: https://www.bip.krakow.pl/zalaczniki/dokumenty/n/283933/karta
Znaleziono link do pobrania: https://www.bip.krakow.pl/plik.php?zid=283933&wer=0&new=t&mode=shw
Pobrano plik: pobrane_pliki\283933_0.pdf
Przetwarzam: https://www.bip.krakow.pl/zalaczniki/dokumenty/n/274429/karta
Znaleziono link do pobrania: https://www.bip.krakow.pl/plik.php?zid=274429&wer=0&new=t&mode=shw
Pobrano plik: pobrane_pliki\274429_0.pdf
Przetwarzam: https://www.bip.krakow.pl/zalaczniki/dokumenty/n/265574/karta
Znaleziono link do pobrania: https://www.bip.krakow.pl/plik.php?zid=265574&we